In [ ]:
#Importing Libraries
import numpy as np
import tensorflow as tf
import pickle #Pickling means it converts the Python objects into byte streams.
 
from tensorflow.keras import layers , activations , models , preprocessing

In [ ]:
a = '/content/new.zip'
!unzip new.zip

y
Archive:  new.zip
replace new/new/data/Acne.yaml? [y]es, [n]o, [A]ll, [N]one, [r]ename:   inflating: new/new/data/Acne.yaml  
replace new/new/data/AIDS.yaml? [y]es, [n]o, [A]ll, [N]one, [r]ename: yy
y
  inflating: new/new/data/AIDS.yaml  
replace new/new/data/allergy.yaml? [y]es, [n]o, [A]ll, [N]one, [r]ename:   inflating: new/new/data/allergy.yaml  
replace new/new/data/Arthritis.yaml? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: new/new/data/Arthritis.yaml  
replace new/new/data/botprofile.yaml? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: new/new/data/botprofile.yaml  
replace new/new/data/Bronchial Asthma.yaml? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: new/new/data/Bronchial Asthma.yaml  
replace new/new/data/cancer.yaml? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: new/new/data/cancer.yaml  
replace new/new/data/Cervical spondylosis.yaml? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: new/new/data/Cervical spondylosis.yaml  
replace new/n

In [ ]:

import tensorflow as tf
from tensorflow.keras import layers, preprocessing, utils
import yaml
#YAML is a human-readable data-serialization language.
#We used yaml format beacuse it is more readable.
#YAML ..Yet Another Markup Language.
import os
#to find dir path

In [ ]:
dir_path = 'new/new/data'
files_list = os.listdir(dir_path + os.sep)
 
questions = list()
answers = list()
 
for filepath in files_list:
    stream = open( dir_path + os.sep + filepath , 'rb')
    docs = yaml.safe_load(stream)
    conversations = docs['conversations']
    
    for con in conversations:
       if len( con )> 1:
            questions.append(con[0])
            answers.append(con[1])
print(questions)
 
answers_with_tags = list()
for i in range( len( answers ) ):
    if type( answers[i] ) == str:
        answers_with_tags.append( answers[i] )
    else:
        questions.pop( i )
 
answers = list()
for i in range( len( answers_with_tags ) ) :
    answers.append( '<START> ' + answers_with_tags[i] + ' <END>' )
print(answers)
 
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts( questions + answers ) #Updates internal vocabulary based on a list of texts.
VOCAB_SIZE = len( tokenizer.word_index )+1
print( 'VOCAB SIZE : {}'.format( VOCAB_SIZE ))

['What are the symptoms of Malaria', 'What are the precautions taken for Malaria', 'What are the medicines for Malaria', 'Hi doctor.', "I'm not feeling well", 'I have a fever', 'Yes.', 'Hello', 'Hi', 'Greetings!', 'Hello', 'Hi, How is it going?', 'Hi, How is it going?', 'Hi, How is it going?', 'Hi, How is it going?', 'Hi, How is it going?', 'Hi, How is it going?', 'How are you doing?', 'How are you doing?', 'How are you doing?', 'Nice to meet you.', 'How do you do?', 'How do you do?', 'Hi, nice to meet you.', 'It is a pleasure to meet you.', 'Top of the morning to you!', 'Top of the morning to you!', "What's up?", "What's up?", "What's up?", "What's up?", "What's up?", 'byee', 'nice to meet you', 'byee', 'What are the symptoms of Hepatitis C', 'What are the precautions taken for Hepatitis C', 'What are the medicines for Hepatitis C', 'What are the symptoms of Common Cold', 'What are the precautions taken for Common Cold', 'What are the medicines for Common Cold', 'Good morning. I need 

In [ ]:
from gensim.models import Word2Vec
import re

vocab=[]
for word in tokenizer.word_index:
    vocab.append(word)
x = max(vocab, key=len)
print(x)
print(vocab.index(x))


def tokenize(sentences):
    tokens_list=[]
    vocabulary=[]
    for sentence in sentences:
        sentence = sentence.lower()
        #sentence = re.sub( '^0-9a-zA-Z',' ',sentence)
        tokens = sentence.split()
        vocabulary+=tokens
        tokens_list.append(tokens)
       
    return tokens_list,vocabulary
    

p=tokenize(questions+answers)
model = Word2Vec(p[0])

tokenized_questions = tokenizer.texts_to_sequences(questions)
maxlen_questions = max([len(x) for x in tokenized_questions])
padded_questions = preprocessing.sequence.pad_sequences(tokenized_questions, maxlen=maxlen_questions, padding='post')
encoder_input_data = np.array( padded_questions )
print(encoder_input_data.shape, maxlen_questions)

tokenized_answers = tokenizer.texts_to_sequences(answers)
maxlen_answers = max([len(x) for x in tokenized_answers])
padded_answers = preprocessing.sequence.pad_sequences(tokenized_answers, maxlen=maxlen_answers, padding='post')
decoder_input_data = np.array(padded_answers)
print(decoder_input_data.shape, maxlen_answers)

tokenized_answers = tokenizer.texts_to_sequences(answers)
for i in range(len(tokenized_answers)):
    tokenized_answers[i]=tokenized_answers[i][1:]
padded_answers = preprocessing.sequence.pad_sequences(tokenized_answers, maxlen=maxlen_answers, padding='post')
one_hot_answers = utils.to_categorical(padded_answers, VOCAB_SIZE)
decoder_output_data = np.array(one_hot_answers)
print(decoder_output_data.shape)


thiazolidinediones
929
(312, 54) 54
(312, 55) 55
(312, 55, 1134)


In [ ]:
encoder_inputs = tf.keras.layers.Input(shape=( None , ))
encoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 929 , mask_zero=True ) (encoder_inputs)
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM( 929 , return_state=True )( encoder_embedding )
encoder_states = [ state_h , state_c ]

decoder_inputs = tf.keras.layers.Input(shape=( None ,  ))
decoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 929 , mask_zero=True) (decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM( 929 , return_state=True , return_sequences=True )
decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states )
decoder_dense = tf.keras.layers.Dense( VOCAB_SIZE , activation=tf.keras.activations.softmax ) 
output = decoder_dense ( decoder_outputs )

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output )
model.compile(optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 929)    1053486     input_3[0][0]                    
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, None, 929)    1053486     input_4[0][0]                    
____________________________________________________________________________________________

In [ ]:
model.fit([encoder_input_data , decoder_input_data], decoder_output_data, batch_size=50, epochs=50 ) 
model.save( 'model.h5' )

Epoch 1/50
7/7 [==============================] - 57s 7s/step - loss: 1.3388 - accuracy: 0.1246
Epoch 2/50
7/7 [==============================] - 50s 7s/step - loss: 1.1469 - accuracy: 0.1327
Epoch 3/50
7/7 [==============================] - 50s 7s/step - loss: 1.0634 - accuracy: 0.1530
Epoch 4/50
7/7 [==============================] - 50s 7s/step - loss: 1.0404 - accuracy: 0.1974
Epoch 5/50
7/7 [==============================] - 50s 7s/step - loss: 0.9927 - accuracy: 0.1998
Epoch 6/50
7/7 [==============================] - 50s 7s/step - loss: 0.9686 - accuracy: 0.2007
Epoch 7/50
7/7 [==============================] - 50s 7s/step - loss: 0.9456 - accuracy: 0.2025
Epoch 8/50
7/7 [==============================] - 50s 7s/step - loss: 0.9205 - accuracy: 0.2132
Epoch 9/50
7/7 [==============================] - 50s 7s/step - loss: 0.8938 - accuracy: 0.2267
Epoch 10/50
7/7 [==============================] - 50s 7s/step - loss: 0.8641 - accuracy: 0.2327
Epoch 11/50
7/7 [======================

In [ ]:
def make_inference_models():
    
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    
    decoder_state_input_h = tf.keras.layers.Input(shape=( 929 ,))
    decoder_state_input_c = tf.keras.layers.Input(shape=( 929 ,))
    
    
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding , initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = tf.keras.models.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
    return encoder_model , decoder_model

In [ ]:
def str_to_tokens( sentence : str ):
    words = sentence.lower().split()
    tokens_list = list()
    for word in words:
        tokens_list.append( tokenizer.word_index[ word ] ) 
    return preprocessing.sequence.pad_sequences( [tokens_list] , maxlen=maxlen_questions , padding='post')

In [ ]:
enc_model , dec_model = make_inference_models()

for _ in range(10):
    states_values = enc_model.predict( str_to_tokens( input( 'You : ' ) ) )
    empty_target_seq = np.zeros( ( 1 , 1 ) )
    empty_target_seq[0, 0] = tokenizer.word_index['start']
    stop_condition = False
    decoded_translation = ''
    while not stop_condition :
        dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + states_values )
        sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
        sampled_word = None
        for word , index in tokenizer.word_index.items() :
            if sampled_word_index == index :
                decoded_translation += ' {}'.format( word )
                sampled_word = word
        
        if sampled_word == 'end' or len(decoded_translation.split()) > maxlen_answers:
            stop_condition = True
            
        empty_target_seq = np.zeros( ( 1 , 1 ) )  
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        states_values = [ h , c ] 

    print( decoded_translation )